## Mining Concept
- Every user will have a **randomized acceptable word list**
- This word list will have three classes of words
1. Common words
2. Symbol Sequence
3. Number Sequence

<p> 
When a user want to mine a block, the user will open a ticket in the server
and after the ticket is open, the user will start to mine the block
</p> 

<p>
The mine will depend of the contract signatures that will be formed by the server using the word list of the user
</p>


## Steps to mine a block:

1. The user will open a TICKET in the server
2. The server will send a message to the user to start mining if the POOL is open
3. The users will load the RANDOM WORD LIST provided by the server API
4. The user have to use 1 common word, 1 symbol sequence and 1 number sequence, randomize the characters
5. Transform the chartacters in a SHA256 hash
6. Transform the block info in before ticket opened in a SHA256 hash
5. Transform (Chars SHA256 hash + block info hash) in a SHA256 hash - or - send only the chars SHA256 hash
6. If the hash is valid, the user will send the hash to the server
7. The server will check if the hash is valid
8. If the hash is valid, check if the same transactions have the same block state confirmation
9. If this transactions was not confirmed at this point at block history the server will add the user signature to the transaction contract
10. The block only can be hashed when the minimum number of contracts valids with minimum number of signatures is reached
11. The server will use the signature timeline to determine what transactions will be added to the block
12. The server will calculate the master hash after confirm all valid contracts signatures
13. The server will add the block to the blockchain and create a new one with last block unconfirmed transactions but completely unsigned
13. The server will start a open/closed pool cycle
14. The server will send a message to the user to start mining when the POOL is open and user has a ticket
<p>
The timestamps of the signatures will be usefull to version the block, checking it as a timeline
</p>

**The miners will be rewarded with the block reward distributed by the number of signatures**


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import hashlib
import itertools
import requests as re

In [2]:
key = '9e8b3f4bf92577861e06e3bc84a809794dd6a7ab9da1070a78eb763b3399595a'
secret = 'a239c12ecc42911ecbc3b23a9539976caaa60f7d40028dea17cb67cc538c21bd'

headers = {
    'X-API-KEY': key,
    'X-API-SECRET': secret,
}


transactions = re.get('http://localhost/api/v1/tr_to_mine', headers=headers)

transactions = str(transactions.json())

transactions_hash = hashlib.sha256(transactions.encode('utf-8')).hexdigest()

print(transactions_hash)

accepted_sequences = re.get('http://localhost/api/v1/info_to_mine', headers=headers)

accepted_sequences = accepted_sequences.json()

accepted_sequences

def parse_accepted_sequences(accepted_sequences):
    words = accepted_sequences['words']
    numbers = accepted_sequences['number_sequences']
    symbols = accepted_sequences['symbol_sequences']
    return words, numbers, symbols

words, numbers, symbols = parse_accepted_sequences(accepted_sequences)

words, numbers, symbols

551e86307dfaddeb02f0326ec1667eac3257817b41ae8916531a06fbcdf9148c


(['her', 'number', 'it', 'at'],
 ['0654', '1523', '2307', '5890'],
 ['@(#$¨@#¨$@¨$$¨@(#$)', '$#$¨@*#$¨$(@#)', '@#$%^&*()_', ')@#$%^&*('])

In [3]:
def all_combinations(acceptable_words, acceptable_number_sequences, acceptables_symbol_sequences):
    combinations = []
    for comb in itertools.product(acceptable_words, acceptable_number_sequences, acceptables_symbol_sequences):
        combinations.append(''.join(comb))
    return combinations

def hash_combinations(combinations, transactions_hash, run_on=0):
    hash_list = []
    counter = 0
    for combination in combinations:
        for comb in itertools.permutations(combination):
            counter += 1
            if counter > run_on:
                random_hash = hashlib.sha256(''.join(comb).encode('utf-8')).hexdigest()
                hash_list.append(hashlib.sha256((transactions_hash + random_hash).encode('utf-8')).hexdigest())
            if len(hash_list) == 1000000:
                return hash_list

In [4]:
def calculate_hash_rate(acceptable_w, acceptable_n, acceptables_s, transactions_hash, run_on=0):
    start = datetime.datetime.now()
    all_combs = all_combinations(acceptable_w, acceptable_n, acceptables_s)
    hash_list = hash_combinations(all_combs, transactions_hash)
    end = datetime.datetime.now()
    total_seconds = (end - start).total_seconds()
    hash_rate = len(hash_list) / total_seconds
    return hash_rate, start, end, total_seconds, hash_list

def benchmark_table(hash_rate, start, end, total_seconds, hash_list):
    hash_list_table = pd.DataFrame({
        'hash': hash_list,
    })
    hash_rate_table = pd.DataFrame({
        'Hash rate': [hash_rate, hash_rate / 1000, hash_rate / 1000000, hash_rate / 1000000000],
        'Unit': ['h/s', 'kh/s', 'Mh/s', 'Gh/s']
    })
    time_table = pd.DataFrame({
        'Time': [start, end, total_seconds],
        'Unit': ['start', 'end', 'total_seconds'],
    })
    return hash_rate_table, time_table, hash_list_table

In [ ]:
hash_rate, start, end, total_seconds, hash_list = calculate_hash_rate(words, numbers, symbols, transactions_hash)
hash_rate_table, time_table, hash_list_table = benchmark_table(hash_rate, start, end, total_seconds, hash_list)

In [ ]:
hash_list_table

In [ ]:
time_table

In [ ]:
hash_rate_table

In [ ]:
def plot_hash_rate(hash_rate_table):
    hash_rate_table.plot(x='Unit', y='Hash rate', kind='bar', logy=True)
    plt.show()

plot_hash_rate(hash_rate_table)

In [10]:
hash_to_send = '99893b819abfa80498f244fddd080f570cecdc994199d5c6a8148141401cdff1'

params = { 'user_confirmation_hash' : hash_to_send }
confirm_request = re.post('http://localhost/api/v1/confirm_block', headers=headers, params=params)

confirm_request.json()

{'message': 'Contract assigned'}